###  품목유효기간조회
* 품목유효기간조회(메인페이지 → 의약품등 정보 → 품목갱신정보 → 품목유효기간조회) https://nedrug.mfds.go.kr/pbp/CCBBL01

In [ ]:
import pandas as pd

In [ ]:
url = "https://nedrug.mfds.go.kr/pbp/CCBBL01"

In [ ]:
# 전체 URL을 출력해 봅니다.
url = "https://nedrug.mfds.go.kr/pbp/CCBBL01/getList?totalPages=76&page=2&limit=10&sort=&sortOrder=&searchYn=&itemName=&entpName=%EB%8C%80%EC%9B%85&itemSeq="

### 검색 값 설정하기
* 웹사이트에서 검색한 URL 안에 우리의 검색어가 들어가 있습니다.
* 이렇게 검색어를 URL에 붙여서 넘기는 문구를 쿼리스트링이라고 부릅니다.
* 이 문자열을 검색어에 맞게 변경해 줍니다.

In [ ]:
# URL 인코딩 - 한글을 URL에 그대로 보낼 수 없기 때문에 인코딩을 합니다.
# 웹사이트에서 검색한 내용이 인코딩 되어 보이는 것과 같은 현상입니다.
#  parse.quote_plus 를 통해 검색어를 인코딩 합니다.
from urllib import parse
search_word = "셀트리온"
# search_word_encode
# parse.quote_plus(search_word) == "%EB%8C%80%EC%9B%85"
search_word_encode = parse.quote_plus(search_word)
search_word_encode

In [ ]:
# URL을 생성합니다.
page_no = 3
# base_url
# params
base_url = "https://nedrug.mfds.go.kr/pbp/CCBBL01/getList?"
params = f"totalPages=76&page={page_no}&limit=10&sort=&sortOrder=&searchYn=true"
params = f"{params}&itemName=&entpName={search_word_encode}&itemSeq="
params

### base_url 과 검색어가 들어간 파라메터 합치기

In [ ]:
# base_url 과 params 를 합쳐 url을 만듭니다.
# url
url = base_url + params
url

### read_html 로 웹사이트의 내용 읽어오기
* html 소스코드 내부에 < table > 로 된 태그가 있을 때만 가능합니다.

In [ ]:
# read_html을 통해 특정 url에 있는 table 태그에 있는 데이터를 가져옵니다.
table = pd.read_html(url, encoding="utf-8")
len(table)

In [ ]:
table[0]

### 페이지번호, 검색어를 넘기면 데이터를 가져오는 함수 만들기

In [ ]:
# try, except 를 사용해서 오류가 났을 때 오류대신 메시지만 출력하도록 합니다.
#     try:
#     except Exception as e:
#         return e
def get_page_content(page_no, search_word):
    try:
        search_word_encode = parse.quote_plus(search_word)

        base_url = "https://nedrug.mfds.go.kr/pbp/CCBBL01/getList?"
        params = f"totalPages=8&page={page_no}&limit=100&sort=&sortOrder=&searchYn=true"
        params = f"{params}&itemName=&entpName={search_word_encode}&itemSeq="

        url = base_url + params
        table = pd.read_html(url, encoding="utf-8")

        return table[0]
    except Exception as e:
        return e

In [ ]:
# 함수 확인하기
page_no = 3
search_word = "한미약품"
get_page_content(page_no, search_word)

### 반복문을 통해 전체 페이지를 수집

* tqdm은?
    * 오래 걸리는 작업의 진행 상태를 표시합니다.
    * https://tqdm.github.io/

In [ ]:
# tqdm 의 trange를 불러옵니다.
from tqdm import trange
# 페이지마다의 목록을 담아줄 리스트 초기화
page_list = []

# 반복문을 통해 여러 페이지의 목록 수집하기
for page_no in trange(1, 9):
    temp = get_page_content(page_no, search_word)
    page_list.append(temp)

In [ ]:
len(page_list)

### 수집한 데이터 하나의 데이터프레임으로 합치기


<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [ ]:
# concat을 통해 데이터프레임 하나로 만들기
df = pd.concat(page_list)
df.shape

### 파일로 저장하기

In [ ]:
# 저장할 파일명 만들기
file_name = f"{search_word}_품목유효기간.csv"
file_name

In [ ]:
# to_csv 로 저장하기
df.to_csv(file_name, index=False, encoding="cp949")

In [ ]:
# 잘 저장되었는지 확인하기
pd.read_csv(file_name, encoding="cp949")